In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from datetime import datetime
from http.client import responses
import json
import numpy as np
import os
import pandas as pd
import pygsheets
import re
import requests
from tqdm import tqdm


pd.set_option('display.max_colwidth', -1)

In [2]:
url_root_sheet = "https://docs.google.com/spreadsheets/d/1IyfSPiiR0DUxYuConiYzuokM2hUQbOBpuJAeAlGi7kc/edit#gid=223389382"

In [3]:
with open("data/data.json", "r") as f:
    processed = json.load(f)

df_root = pd.read_csv(processed[url_root_sheet] + "/root.csv", sep=";")
df_root

,ID,Level,Program,Year,Module,Discipline,Teacher,URL,Row,Type,IsParsed,ErrorName,Text
0,1,Бакалавриат,Программная инженерия,1,"1,2,3,4",Математический анализ,Федотов,https://drive.google.com/drive/folders/1n-ykGVz7eHWG_PgG0RSo-tBcGs_9s3xS?usp=sharing,3,Drive Directory,NaN,NaN,https://drive.google.com/drive/folders/1n-ykGVz7eHWG_PgG0RSo-tBcGs_9s3xS?usp=sharing
1,2,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по матанализу на Python,Шайхелисламов Д.С.,https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=0&fvid=982303739,4,Google Sheet,NaN,NaN,https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=0&fvid=982303739 https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=1790524453
2,3,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по матанализу на Python,Шайхелисламов Д.С.,https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=1790524453,4,Google Sheet,NaN,NaN,https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=0&fvid=982303739 https://docs.google.com/spreadsheets/d/1QwLYiokR_V_eYYzpKkOedOQaPY4A07HyzrqjGJgAQ6k/edit#gid=1790524453
3,4,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по математическому анализу в среде Matlab,Макаров С.Л.,http://serjmak.com/2students/matlaba/,5,NaN,False,Internal Server Error,http://serjmak.com/2students/matlaba/
4,5,Бакалавриат,Программная инженерия,1,"1,2,3,4",Алгебра,Чернышев Всеволод Леонидович,https://docs.google.com/spreadsheets/d/1fZEpApQu46aBq_1OO9C03ttjyLWhS_MH_G3SGKvf12A/edit#gid=553615486,6,Google Sheet,NaN,NaN,https://docs.google.com/spreadsheets/d/1fZEpApQu46aBq_1OO9C03ttjyLWhS_MH_G3SGKvf12A/edit#gid=553615486
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,343,Бакалавриат,Компьютерные науки и анализ данных,1,3,Программирование на С++,Леванов Дмитрий Александрович,https://docs.google.com/spreadsheets/d/1DC7xSS-2ycn-y3cSsegnFpms-wChg3dXMb4DGgE4ZZo/edit#gid=276600795,267,Google Sheet,NaN,NaN,https://docs.google.com/spreadsheets/d/1DC7xSS-2ycn-y3cSsegnFpms-wChg3dXMb4DGgE4ZZo/edit#gid=276600795
343,344,Бакалавриат,Компьютерные науки и анализ данных,1,4,Инструменты промышленной разработки,Горшков Сергей Сергеевич,NaN,268,NaN,False,URL invalid,NaN
344,345,Бакалавриат,Компьютерные науки и анализ данных,1,3-4,Правовая грамотность,Хабибулин Тагир Маратович,NaN,269,NaN,False,URL invalid,NaN
345,346,Бакалавриат,Компьютерные науки и анализ данных,1,1-4,АЯ для ОиК целей,"Журавлева Анна Петровна, Сафонова Юлия Владимировна",NaN,270,NaN,False,URL invalid,SmartLMS


In [4]:
# Недоступные ссылки (либо отказ в доступе, либо ведомость удалена, либо еще что-то)
df_root[pd.notna(df_root.ErrorName)]

,ID,Level,Program,Year,Module,Discipline,Teacher,URL,Row,Type,IsParsed,ErrorName,Text
3,4,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по математическому анализу в среде Matlab,Макаров С.Л.,http://serjmak.com/2students/matlaba/,5,NaN,False,Internal Server Error,http://serjmak.com/2students/matlaba/
7,8,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по алгебре в среде Matlab,Макаров С.Л.,http://serjmak.com/2students/matlaba/,8,NaN,False,Internal Server Error,http://serjmak.com/2students/matlaba/
24,25,Бакалавриат,Программная инженерия,1,4,Основы программирования на С++,Чуйкин Н.К.,NaN,15,NaN,False,URL invalid,NaN
29,30,Бакалавриат,Программная инженерия,1,"1,2,3","Научно-исследовательский семинар ""Нейроинформатика”",Харламов А.А.,NaN,20,NaN,False,URL invalid,NaN
30,31,Бакалавриат,Программная инженерия,1,"1,2,3","Научно-исследовательский семинар ""Нейросетевые технологии""",Пантюхин Д.В.,https://eduhseru.sharepoint.com/:x:/s/msteams_7b6a7e/EfdfzXciRadIqjm9nkjavmYBEPV0au-lvGV0mZtNtdJU9w,21,Sharepoint,False,Forbidden,https://eduhseru.sharepoint.com/:x:/s/msteams_7b6a7e/EfdfzXciRadIqjm9nkjavmYBEPV0au-lvGV0mZtNtdJU9w
...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,338,Бакалавриат,Компьютерные науки и анализ данных,1,1-4,Математический анализ,Лобода Артем Александрович,NaN,263,NaN,False,URL invalid,NaN
341,342,Бакалавриат,Компьютерные науки и анализ данных,1,3,Python для анализа данных,Горшков Сергей Сергеевич,NaN,266,NaN,False,URL invalid,NaN
343,344,Бакалавриат,Компьютерные науки и анализ данных,1,4,Инструменты промышленной разработки,Горшков Сергей Сергеевич,NaN,268,NaN,False,URL invalid,NaN
344,345,Бакалавриат,Компьютерные науки и анализ данных,1,3-4,Правовая грамотность,Хабибулин Тагир Маратович,NaN,269,NaN,False,URL invalid,NaN


In [5]:
# Какие ошибки были при парсинге ссылок
df_root.ErrorName.unique()

array([nan, 'Internal Server Error', 'URL invalid', 'Forbidden',
       'Unauthorized', 'Gone'], dtype=object)

In [6]:
# Какие ведомости были удалены
df_root[df_root.ErrorName == "Gone"]

,ID,Level,Program,Year,Module,Discipline,Teacher,URL,Row,Type,IsParsed,ErrorName,Text
56,57,Бакалавриат,Программная инженерия,3,"1,2",Математические методы анализа данных (преподается на английском языке),Кузина Анна,https://docs.google.com/spreadsheets/d/1Xf7hJfB5M28uLB7A7NLtJDR5A_xHrl-uvwiT1NnqKGc/edit?usp=sharing,42,Google Sheet,False,Gone,https://docs.google.com/spreadsheets/d/1Xf7hJfB5M28uLB7A7NLtJDR5A_xHrl-uvwiT1NnqKGc/edit?usp=sharing
76,77,Бакалавриат,Программная инженерия,3,"1,2",Разработка IDE,Васильев Андрей Дмитриевич,https://docs.google.com/spreadsheets/d/1-vb7h-hIuPaWt6XnQa6ql3wXA5kAWmTgVctD5Jrte5A/edit?usp=sharing,61,Google Sheet,False,Gone,https://docs.google.com/spreadsheets/d/1-vb7h-hIuPaWt6XnQa6ql3wXA5kAWmTgVctD5Jrte5A/edit?usp=sharing
126,127,Бакалавриат,Прикладной анализ данных,2,"1,2,3,4",Теория вероятностей и математическая статистика,Лукьянченко Петр Павлович,https://docs.google.com/spreadsheets/d/1qY2_yFbiBpG_ggxWhYlank1JLQGdfyTqPQ1mafvfufE/edit#gid=0,102,Google Sheet,False,Gone,https://docs.google.com/spreadsheets/d/1qY2_yFbiBpG_ggxWhYlank1JLQGdfyTqPQ1mafvfufE/edit#gid=0
141,142,Бакалавриат,Прикладной анализ данных,3,"1,2,3,4",Временные ряды и случайные процессы,Лукьянченко Петр Павлович,https://docs.google.com/spreadsheets/d/1Md5d_1X4R7OhJP0mqtPKvSDg0UWpaLpsuzy_LPbzF1M/edit#gid=0,112,Google Sheet,False,Gone,https://docs.google.com/spreadsheets/d/1Md5d_1X4R7OhJP0mqtPKvSDg0UWpaLpsuzy_LPbzF1M/edit#gid=0
162,163,Бакалавриат,Прикладная математика и информатика,1,"1,2,3,4",Английский язык,NaN,https://docs.google.com/spreadsheets/d/1HPNNcGqUnNnHxP_kg8VdoOkZLnZnJBrE/edit#gid=517328135,133,Google Sheet,False,Gone,https://docs.google.com/spreadsheets/d/1HPNNcGqUnNnHxP_kg8VdoOkZLnZnJBrE/edit#gid=517328135 https://docs.google.com/spreadsheets/d/1cKuEw-WSncFSdlykT4iJsk6YaZXrUhAK/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1t3cVdFVgIPu2EGJV7m7hnkWwTOavIUqh/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1ObWOxbPd_wCrcDamvEfvIT5an4hLoKRr/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/147zAGU0HGaoAboFs0C583bjCmVKZEUPA/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1ENPqJFyrPGpFMBvLiYfSx4mh06Ikpl5D/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1Yr5XJFxaZnpxW-GjLk-J9ChMGmYHuH0S/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1Q0KCmtSoiix09DD6m_GWghKW8YoAJ2df/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1hss8s7iK1Qf-h0ZaIOsNmHHqcqWKlbjl/edit#gid=517328135 https://docs.google.com/spreadsheets/d/1xWr76SAJcITJlaMGi8SXsbgAxwO1C7Pp/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/17RPF236hBxcD_sjbFbLYOKInhBfFb4hq/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1b5aQe70i3AOXweK-Hx0-dQ77zqcyl4AD/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://docs.google.com/spreadsheets/d/1jVgGctbHrX2qHftykz4ljIVvoW9I1jeq/edit?usp=sharing&ouid=105703278948485537440&rtpof=true&sd=true https://drive.google.com/file/d/1j2fm0auv2Ct5gCYqR0-mX1l8RmmGPEON/view?usp=sharing https://drive.google.com/file/d/10_2D0QZlCtrIb-iz8mwBThei3qEcWn28/view?usp=sharing https://docs.google.com/spreadsheets/d/1_JPBDMp0HjzGtxM6RTfqp2oTb3-u997P/edit#gid=1209800553
199,200,Бакалавриат,Прикладная математика и информатика,1,"1,2,3",Дискретная математика (пилот),Подольский Владимир Владимирович,https://docs.google.com/spreadsheets/d/1S5XMkwGeDDpLvkNtTazY9YW-OMZEBKeXWqyJ_agaUKI/edit#gid=0,138,Google Sheet,False,Gone,211 https://docs.google.com/spreadsheets/d/1Liui4kmOC0RPQeZUMJGYTxdjMdruMjv1sxWdVyZPHGU/edit#gid=0 212 https://docs.google.com/spreadsheets/d/1zYnqIdq

In [7]:
print("Валидных открытых ведомостей всего {} из {}".format(len(df_root[pd.isna(df_root.ErrorName)]), len(df_root)))

Валидных открытых ведомостей всего 284 из 347


In [8]:
print("Форматы ведостей")
print(df_root.Type.unique())

Форматы ведостей
['Drive Directory' 'Google Sheet' nan 'Drive PDF' 'Sharepoint' 'Dropbox']


In [9]:
# Ведомости, которые не были предоставлены
df_root[pd.isna(df_root.URL)]

,ID,Level,Program,Year,Module,Discipline,Teacher,URL,Row,Type,IsParsed,ErrorName,Text
24,25,Бакалавриат,Программная инженерия,1,4,Основы программирования на С++,Чуйкин Н.К.,NaN,15,NaN,False,URL invalid,NaN
29,30,Бакалавриат,Программная инженерия,1,"1,2,3","Научно-исследовательский семинар ""Нейроинформатика”",Харламов А.А.,NaN,20,NaN,False,URL invalid,NaN
33,34,Бакалавриат,Программная инженерия,1,"1,2,3,4",факультатив Английский язык,"ШИЯ, менеджер Котрина Елена",NaN,24,NaN,False,URL invalid,NaN
47,48,Бакалавриат,Программная инженерия,2,"1,2,3","Научно-исследовательский семинар ""Методы и алгоритмы защиты информации""",Аржанцев И.В.,NaN,33,NaN,False,URL invalid,NaN
55,56,Бакалавриат,Программная инженерия,2,"1,2,3",Английский язык,"ШИЯ, менеджер Котрина Елена",NaN,41,NaN,False,URL invalid,NaN
65,66,Бакалавриат,Программная инженерия,3,"1,2",Прикладные методы линейной алгебры (преподается на английском языке),Пионтковский Д.И.,NaN,50,NaN,False,URL invalid,NaN
69,70,Бакалавриат,Программная инженерия,3,"3,4",Распределенные вычисления (преподается на английском языке),Панфилов П.Б.,NaN,54,NaN,False,URL invalid,NaN
73,74,Бакалавриат,Программная инженерия,3,"3,4",UX-исследования для создания новых цифровых продуктов (преподается на английском языке),Иванов В.Д.,NaN,58,NaN,False,URL invalid,NaN
74,75,Бакалавриат,Программная инженерия,3,"1,2",Верификация программ,Щепетков Илья Викторович,NaN,59,NaN,False,URL invalid,NaN
75,76,Бакалавриат,Программная инженерия,3,"3,4",Компиляторные технологии,С.С.Гайсарян,NaN,60,NaN,False,URL invalid,NaN


In [10]:
# Ведомости, у которых НЕ определен формат и валидна (и доступна) ссылка на ведомость
df_root[(pd.isna(df_root.Type)) & (pd.notna(df_root.URL))]

,ID,Level,Program,Year,Module,Discipline,Teacher,URL,Row,Type,IsParsed,ErrorName,Text
3,4,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по математическому анализу в среде Matlab,Макаров С.Л.,http://serjmak.com/2students/matlaba/,5,NaN,False,Internal Server Error,http://serjmak.com/2students/matlaba/
7,8,Бакалавриат,Программная инженерия,1,"2,3",Компьютерный практикум по алгебре в среде Matlab,Макаров С.Л.,http://serjmak.com/2students/matlaba/,8,NaN,False,Internal Server Error,http://serjmak.com/2students/matlaba/
10,11,Бакалавриат,Программная инженерия,1,"1,2,3",Программирование на C#,Дударев В.А.,https://edu.hse.ru/grade/report/grader/index.php?id=105244,9,NaN,NaN,NaN,https://docs.google.com/spreadsheets/d/1-QFsZNikCjVGx7rwRFq1FncGM5c-lUpukEmLv009NlI/edit#gid=1150291018 (1 модуль) https://drive.google.com/drive/folders/1-wn4IV22GNIekW7_nhl_wBzHVdr6trYJ?usp=sharing (итоговая) https://edu.hse.ru/grade/report/grader/index.php?id=105244
52,53,Бакалавриат,Программная инженерия,2,"1,2,3","Научно-исследовательский семинар ""Разработка облачных и мобильных приложений на платформе Google Android""",Макаров С.Л.,https://tinyurl.com/dkurvaps,38,NaN,NaN,NaN,https://tinyurl.com/dkurvaps
62,63,Бакалавриат,Программная инженерия,3,"3,4",Проектирование архитектуры программных систем,Мицюк А.А.,https://edu.hse.ru/grade/report/index.php?id=111100,47,NaN,NaN,NaN,https://edu.hse.ru/grade/report/index.php?id=111100
64,65,Бакалавриат,Программная инженерия,3,"1,2",Экосистемы интернета вещей (преподается на английском языке),Макаров С.Л.,https://tinyurl.com/3u4f57jh,49,NaN,NaN,NaN,https://tinyurl.com/3u4f57jh
103,104,Бакалавриат,Программная инженерия,4,"1,2,3","Научно-исследовательский семинар ""Тренды облачной и мобильной разработки""",Макаров С.Л.,https://tinyurl.com/upn827mk,88,NaN,NaN,NaN,https://tinyurl.com/upn827mk
109,110,Бакалавриат,Прикладной анализ данных,1,"1,2",Основы права blended,Журавлев Михаил Сергеевич,https://drive.google.com/open?id=1Kid4BcZ2YhTynUD_LUndlgofzoFOzuWK,94,NaN,NaN,NaN,https://drive.google.com/open?id=1Kid4BcZ2YhTynUD_LUndlgofzoFOzuWK
139,140,Бакалавриат,Прикладной анализ данных,3,"1,2,3,4",Машинное обучение 1 (лекции вместе с МИЭФ/ online via Zoom),Мельников Олег,https://drive.google.com/open?id=1zNBo7B7deoZyEDDpxpNF9Q1BFxejDnh0,110,NaN,NaN,NaN,https://drive.google.com/open?id=1zNBo7B7deoZyEDDpxpNF9Q1BFxejDnh0
206,207,Бакалавриат,Прикладная математика и информатика,1,"1,2,3,4",Линейная алгебра и геометрия (пилот),Трушин Дмитрий Витальевич,http://wiki.cs.hse.ru/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0_%D0%B8_%D0%B3%D0%B5%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%8F_%D0%BD%D0%B0_%D0%9F%D0%9C%D0%98_2021/2022_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA)#.D0.92.D0.B5.D0.B4.D0.BE.D0.BC.D0.BE.D1.81.D1.82.D0.B8_.D1.82.D0.B5.D0.BA.D1.83.D1.89.D0.B5.D0.B3.D0.BE_.D0.BA.D0.BE.D0.BD.D1.82.D1.80.D0.BE.D0.BB.D1.8F,140,NaN,NaN,NaN,http://wiki.cs.hse.ru/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0_%D0%B8_%D0%B3%D0%B5%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%8F_%D0%BD%D0%B0_%D0%9F%D0%9C%D0%98_2021/2022_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA)#.D0.92.D0.B5.D0.B4.D0.BE.D0.BC.D0.BE.D1.81.D1.82.D0.B8_.D1.82.D0.B5.D0.BA.D1.83.D1.89.D0.B5.D0.B3.D0.BE_.D0.BA.D0.BE.D0.BD.D1.82.D1.80.D0.BE.D0.BB.D1.8F


In [11]:
with open("config/config.json", "r") as f:
    config = json.load(f)

for url_type in config["URL_TYPE"].values():
    cnt_type = len(df_root[df_root.Type == url_type])
    cnt_type_url = len(df_root[(df_root.Type == url_type) & (pd.notna(df_root.URL))])
    print("{}: valid and accessible {} out of {}".format(url_type, cnt_type_url, cnt_type))
    
cnt_unknown_url = len(df_root[(pd.isna(df_root.Type)) & (pd.notna(df_root.URL))])
cnt_unknown = len(df_root[pd.isna(df_root.Type)])
print("\nValid and accessible of unknown type:", cnt_unknown_url, "out of", cnt_unknown)

Google Sheet: valid and accessible 237 out of 237
Drive Directory: valid and accessible 5 out of 5
Drive PDF: valid and accessible 13 out of 13
Drive Unknown File: valid and accessible 0 out of 0
Sharepoint: valid and accessible 6 out of 6
MS Teams: valid and accessible 0 out of 0
Dropbox: valid and accessible 10 out of 10

Valid and accessible of unknown type: 26 out of 76


In [12]:
import math

for url_type in config["URL_TYPE"].values():
    cnt = len(df_root[df_root.Type == url_type])
    print("{} ~ {} %".format(url_type, math.ceil(100 * cnt / len(df_root))))

cnt = len(df_root[pd.isna(df_root.Type)])
print("\nOther types ~", math.ceil(100 * cnt / len(df_root)), "%")

Google Sheet ~ 69 %
Drive Directory ~ 2 %
Drive PDF ~ 4 %
Drive Unknown File ~ 0 %
Sharepoint ~ 2 %
MS Teams ~ 0 %
Dropbox ~ 3 %

Other types ~ 22 %


In [13]:
for error in df_root.ErrorName.unique():
    if pd.notna(error):
        cnt = len(df_root[df_root.ErrorName == error])
        print("{}: {}".format(error, cnt))

Internal Server Error: 2
URL invalid: 50
Forbidden: 1
Unauthorized: 3
Gone: 7
